# BLADE DEMO

synopsis
- Load modules
- Example simulation data generation (some variability options; multiple samples)
- Application of BLADE
- Performance evaluation (both cellular fraction/purification)


In [8]:
import sys, os
sys.path.append(os.path.relpath('../python/')) # location of python script

from BLADE import BLADE_framework as BLADE #to be changed
import numpy as np
from numpy import transpose as t
import itertools
import pickle
from scipy.optimize import nnls
from sklearn.svm import SVR
from sklearn.svm import NuSVR
from sklearn.metrics import mean_squared_error as mse
import pandas as pd

## Generation of simulation data 

- Also provide some overview of data (e.g., what is the variability? - both in log-scale and linear scale?)
- Allow users to change setting (e.g., number of cell types and samples)

In [9]:
# training data
Ncells = [5]
Ngenes = [500]
Nsamples = [20]
Noises = [0.25, 0.5, 0.75, 1] #try with less no. of noises [0.1, 0.2, 0.5, 0.75, 1, 1.25, 1.5]


simfile = '../simulationdata.pickle'
if not os.path.exists(simfile):
    Synthetic_data = dict()
else:
    Synthetic_data = pickle.load(open(simfile, 'rb'))

for Ncell, Ngene, Nsample, Noise in itertools.product(
    Ncells, Ngenes, Nsamples, Noises
            ):
    name = str(Ncell) +'_'+ str(Ngene) +'_'+ str(Nsample) +'_'+ str(Noise)
    
    if not name in list(Synthetic_data.keys()):
        Coef = np.random.dirichlet(np.ones(Ncell)*5, Nsample).transpose()
        Mu_train = np.random.normal(0, 2, size=(Ngene,Ncell))
        Omega_train = np.ones((Ngene,Ncell)) * Noise

        X_train = np.random.normal(Mu_train, Omega_train, size=(Nsample, Ngene, Ncell))
        Y_train = np.zeros((Ngene, Nsample))
        for i in range(Nsample):
            Y_train[:,i] = np.log(np.dot(np.exp(X_train[i,:,:]), Coef[:,i])+1)

    
        Synthetic_data[name] = {
            'Coef': Coef,
            'Mu' : Mu_train, #signature
            'X' : X_train, #highres
            'Y' : Y_train,
            'Omega' : Omega_train
        }

        
with open(simfile, 'wb') as fp:
    pickle.dump(Synthetic_data, fp, protocol=pickle.HIGHEST_PROTOCOL)
    


### BLADE application
- configuration of parameters
- running BLADE

In [10]:
pars = {
    'Alpha': [1],
    'Alpha0': [0.1, 0.5, 1],
    'Kappa0': [1, 0.5, 0.1],
    'SigmaY': [0.1, 0.5]
}
Nrep=3
Nrepfinal=5
Njob=10

In [11]:
for Ncell, Ngene, Nsample, Noise in itertools.product(
    Ncells, Ngenes, Nsamples, Noises,
                ):

    name = str(Ncell) +'_'+ str(Ngene) +'_'+ str(Nsample) +'_'+ str(Noise)
    outfile = '../data/BLADE_outcome_' + name + '.pickle'

    print('creating ' + outfile)
    Y = Synthetic_data[name]['Y']
    mean = Synthetic_data[name]['Mu']
    sd = Synthetic_data[name]['Omega']
    
    Ind_sample = [True]*5 + [False]*(Nsample - 5)
    Marker_Index = [True] * Ngene
    
    final_obj, best_obj, best_set, outs = BLADE(
            mean, sd, np.exp(Y)-1, Marker_Index, Ind_sample,
            pars['Alpha'], pars['Alpha0'], pars['Kappa0'], pars['SigmaY'],
            Nrep=Nrep, Njob=Njob, Nrepfinal=Nrepfinal, fsel=0)
        
    pickle.dump(
            {
                'final_obj': final_obj,
                'best_obj': best_obj,
                'best_set': best_set,
                'outs' : outs,
                'pars' : pars
            },
            open(outfile, 'wb')
    )
    
#37min

creating ../data/BLADE_outcome_5_500_20_0.25.pickle
all of 500 genes are used for optimization.
Number of samples used: 5 out of 20 samples.
Initialization with Support vector regression


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   4 out of  20 | elapsed:    2.9s remaining:   11.4s
[Parallel(n_jobs=10)]: Done   7 out of  20 | elapsed:    4.5s remaining:    8.3s
[Parallel(n_jobs=10)]: Done  10 out of  20 | elapsed:    4.6s remaining:    4.6s
[Parallel(n_jobs=10)]: Done  13 out of  20 | elapsed:    5.0s remaining:    2.7s
[Parallel(n_jobs=10)]: Done  16 out of  20 | elapsed:    5.7s remaining:    1.4s
[Parallel(n_jobs=10)]: Done  20 out of  20 | elapsed:    6.9s finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.


No feature filtering is done (fsel = 0)


[Parallel(n_jobs=10)]: Done   5 tasks      | elapsed:  1.6min
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:  3.2min
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:  4.6min
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:  5.2min
[Parallel(n_jobs=10)]: Done  41 out of  54 | elapsed:  6.4min remaining:  2.0min
[Parallel(n_jobs=10)]: Done  47 out of  54 | elapsed:  6.9min remaining:  1.0min
[Parallel(n_jobs=10)]: Done  54 out of  54 | elapsed: 10.6min finished


Done optimization, elapsed time (min): 10.634846274058024
Start inferring per-sample gene expression levels using the entire genes and samples


...

[Parallel(n_jobs=10)]: Done   4 out of  20 | elapsed:    0.5s remaining:    1.9s
[Parallel(n_jobs=10)]: Done   7 out of  20 | elapsed:    0.6s remaining:    1.2s
[Parallel(n_jobs=10)]: Done  10 out of  20 | elapsed:    0.9s remaining:    0.9s
[Parallel(n_jobs=10)]: Done  13 out of  20 | elapsed:    0.9s remaining:    0.5s
[Parallel(n_jobs=10)]: Done  16 out of  20 | elapsed:    1.0s remaining:    0.3s
[Parallel(n_jobs=10)]: Done  20 out of  20 | elapsed:    1.3s finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.


No feature filtering is done (fsel = 0)


[Parallel(n_jobs=10)]: Done   5 tasks      | elapsed:   12.5s
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:  1.7min
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:  2.4min
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:  3.5min
[Parallel(n_jobs=10)]: Done  41 out of  54 | elapsed:  4.3min remaining:  1.4min
[Parallel(n_jobs=10)]: Done  47 out of  54 | elapsed:  4.9min remaining:   43.9s
[Parallel(n_jobs=10)]: Done  54 out of  54 | elapsed:  6.0min finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.


Done optimization, elapsed time (min): 6.017038981119792
Start inferring per-sample gene expression levels using the entire genes and samples


[Parallel(n_jobs=10)]: Done   2 out of   5 | elapsed:   35.7s remaining:   53.6s
[Parallel(n_jobs=10)]: Done   3 out of   5 | elapsed:   36.2s remaining:   24.1s
[Parallel(n_jobs=10)]: Done   5 out of   5 | elapsed:   43.3s remaining:    0.0s
[Parallel(n_jobs=10)]: Done   5 out of   5 | elapsed:   43.3s finished


## Application of NNLS 

In [32]:
for Ncell, Ngene, Nsample, Noise in itertools.product(
    Ncells, Ngenes, Nsamples, Noises,
                ):

    name = str(Ncell) +'_'+ str(Ngene) +'_'+ str(Nsample) +'_'+ str(Noise)
    outfile = '../data/NNLS_outcome_' + name + '.pickle'

    if not os.path.exists(outfile):
        
        print('creating ' + outfile)
        Y = Synthetic_data[name]['Y']
        mean = Synthetic_data[name]['Mu']
        
        NNLS_mat = np.zeros(Synthetic_data[name]['Coef'].shape)
        for i in range(Nsample):
            NNLS_mat[:,i] = nnls(np.exp(mean)-1, np.exp(Y[:,i])-1)[0]
        
        pickle.dump(
            {
                'Fraction': NNLS_mat
            },
            open(outfile, 'wb')
        )

creating ../data/NNLS_outcome_5_500_20_0.25.pickle
creating ../data/NNLS_outcome_5_500_20_0.5.pickle
creating ../data/NNLS_outcome_5_500_20_0.75.pickle
creating ../data/NNLS_outcome_5_500_20_1.pickle


In [50]:
for Ncell, Ngene, Nsample, Noise in itertools.product(
    Ncells, Ngenes, Nsamples, Noises,
                ):

    name = str(Ncell) +'_'+ str(Ngene) +'_'+ str(Nsample) +'_'+ str(Noise)
    outfile = '../data/SVR_outcome_' + name + '.pickle'

    if not os.path.exists(outfile):
        
        print('creating ' + outfile)
        Y = Synthetic_data[name]['Y']
        X = Synthetic_data[name]['Mu']
   
        # estimate fraction
        SVRcoef = np.zeros((Ncell, Nsample))
        Selcoef = np.zeros((Ngene, Nsample))
        Nus = [0.25, 0.5, 0.75]
        for i in range(Nsample):
            sols = [NuSVR(kernel='linear', nu=nu).fit(X,Y[:,i]) for nu in Nus]
            RMSE = [mse(sol.predict(X), Y[:,i]) for sol in sols]
            Selcoef[sols[np.argmin(RMSE)].support_, i] = 1
            SVRcoef[:,i] = np.maximum(sols[np.argmin(RMSE)].coef_,0)
            
           
        # estimate per-cell expression
        NNLS_mat = np.zeros((Ngene, Ncell))
        for g in range(Ngene):
            NNLS_mat[g,:] = nnls(np.transpose(SVRcoef), Y[g,:])[0]
        
        pickle.dump(
            {
                'Fraction' : SVRcoef,
                'Signature': NNLS_mat
             },
            open(outfile, 'wb')
         )

creating ../data/SVR_outcome_5_500_20_0.25.pickle
creating ../data/SVR_outcome_5_500_20_0.5.pickle
creating ../data/SVR_outcome_5_500_20_0.75.pickle
creating ../data/SVR_outcome_5_500_20_1.pickle


In [67]:
outcome = pd.DataFrame()
outcome_summary = pd.DataFrame()
outcome_Absolute = pd.DataFrame()

outcome_svr = pd.DataFrame()
outcome_summary_svr = pd.DataFrame()
outcome_Absolute_svr = pd.DataFrame()

for Ncell, Ngene, Nsample, Noise in itertools.product(
    Ncells, Ngenes, Nsamples, Noises,
                ):

    name = str(Ncell) +'_'+ str(Ngene) +'_'+ str(Nsample) +'_'+ str(Noise)
    outfile = '../data/BLADE_outcome_' + name + '.pickle'
    BLADE = pickle.load(open(outfile, 'rb'))
    outfile_SVR = '../data/SVR_outcome_' + name + '.pickle'
    SVR = pickle.load(open(outfile_SVR, 'rb'))
    
    #Loading test data
    Y = np.array(Synthetic_data[name]['Y'])     
    mean = np.array(Synthetic_data[name]['Mu'])
    sd = np.array(Synthetic_data[name]['Omega'])
    X = np.array(Synthetic_data[name]['X'])
    fraction = np.array(Synthetic_data[name]['Coef'])
    
    # Measuring performance of BLADE
    obj = BLADE['final_obj']
    BLADE_Y = np.log(obj.ExpQ(obj.Nu, obj.Beta, obj.Omega))
    BLADE_X = obj.Nu #highres purification
    BLADE_M = np.mean(obj.Nu, 0) #group mode
    BLADE_F = t(obj.ExpF(obj.Beta)) 
        
    #### GROUP MODE BLADE ####
    CorsNu = [np.corrcoef(mean[:,i], BLADE_M[:,i], rowvar=True)[0,1] for i in range(Ncell)]
   
    Cors = [np.corrcoef(BLADE_F[i,:], fraction[i,:], rowvar=True)[0,1] for i in range(Ncell)]
    row = pd.DataFrame(np.array(([name]*Ncell, Cors, CorsNu)).transpose(),
                    columns = ['name', 'Correlation', 'CorrelationNu'])
    outcome = outcome.append(row, ignore_index=True)
    mCors = np.mean(Cors)
    Cors = [np.corrcoef(BLADE_F[:,i], fraction[:,i], rowvar=True)[0,1] for i in range(Nsample)]
    row = pd.DataFrame(np.array(([name]*Nsample, Cors)).transpose(),
                    columns = ['name', 'Correlation'])
    outcome_Absolute = outcome_Absolute.append(row, ignore_index=True)
    row = pd.DataFrame(np.array((name, mCors, np.mean(Cors), np.mean(CorsNu))),
                    index = ['name', 'Correlation', 'AbsCorrelation', 'CorrelationNu'])
    outcome_summary = outcome_summary.append(row.transpose(), ignore_index=True)
    
    #### GROUP MODE SVR ####

    
    # Measuring performance of SVR
    SVR_M = SVR['Signature']
    SVR_F = SVR['Fraction'] 
    
    CorsNu_SVR = [np.corrcoef(SVR_M[:,i], mean[:,i], rowvar=True)[0,1] for i in range(Ncell)] #signature
   
    Cors_SVR = [np.corrcoef(SVR_F[i,:], fraction[i,:], rowvar=True)[0,1] for i in range(Ncell)]
    row_SVR = pd.DataFrame(np.array(([name]*Ncell, Cors_SVR, CorsNu_SVR)).transpose(),
                    columns = ['name', 'Correlation_SVR', 'CorrelationNu_SVR'])
    outcome_svr = outcome_svr.append(row_SVR, ignore_index=True)
    mCors_SVR = np.mean(Cors_SVR)
    Cors_SVR = [np.corrcoef(SVR_F[:,i], fraction[:,i], rowvar=True)[0,1] for i in range(Nsample)]
    row_SVR = pd.DataFrame(np.array(([name]*Nsample, Cors_SVR)).transpose(),
                    columns = ['name', 'Correlation_SVR'])
    outcome_Absolute_svr = outcome_Absolute_svr.append(row, ignore_index=True)
    row_SVR = pd.DataFrame(np.array((name, mCors_SVR, np.mean(Cors_SVR), np.mean(CorsNu_SVR))),
                    index = ['name', 'Correlation_SVR', 'AbsCorrelation_SVR', 'CorrelationNu_SVR'])
    outcome_summary_svr = outcome_summary_svr.append(row_SVR.transpose(), ignore_index=True)
    
    
    #### HIGH RESOLUTION ####
        

In [68]:
outcome_summary_svr

,name,Correlation_SVR,AbsCorrelation_SVR,CorrelationNu_SVR
0,5_500_20_0.25,0.9695633796127192,0.8375456340554231,0.7410775660295024
1,5_500_20_0.5,0.9579902345399336,0.8724566427423751,0.7028124751421891
2,5_500_20_0.75,0.9474632527573463,0.9245186273995089,0.6569402832797028
3,5_500_20_1,0.8936043118122873,0.8862554739781775,0.5404904681290625


In [64]:
outcome_summary

,name,Correlation,AbsCorrelation,CorrelationNu
0,5_500_20_0.25,0.8754087431710029,0.8347977393235728,0.6147385922803164
1,5_500_20_0.5,0.8879910425102059,0.37525846998979195,0.7994345523020032
2,5_500_20_0.75,0.9777040812910036,0.9553118247449653,0.9710773469475434
3,5_500_20_1,0.8681060928283653,0.808611426692601,0.8885837542606441
